## 일단 사용해 봅시다

In [ ]:
!pip install langchain
!pip install openai

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "발급받은 API 키"

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.5)
text = "피보나치 수열을 구하는 Python 프로그램을 작성해주세요."
print(llm(text))

In [ ]:
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

for i in range(10):
    print(fibonacci(i))

## 단독 체인으로 질의하기

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

llm = ChatOpenAI()
prompt = PromptTemplate(
    input_variables=["product"],
    template="Python으로 다음 프로그램 코드에 테스트 코드를 추가하여 작성해주세요.: {product}",
)

chain = LLMChain(llm=llm, prompt=prompt)
chain.run("피보나치 수열을 구한다")

## 단독 체인을 연결해 보기

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

llm = ChatOpenAI()

prompt = PromptTemplate(
    input_variables=["product"],
    template="다음에 제시한 테마의 가치를 정의하고 새로운 비즈니스의 가능성을 개념적으로 800자 이내로 생각해주세요: {product}",
)

chain = LLMChain(llm=llm, prompt=prompt)
idea = chain.run("농업")

print(idea)

In [ ]:
prompt = PromptTemplate(
    input_variables=["idea"],
    template="다음 아이디어로 구체적인 계획을 다섯 개 생각하고, IT가 효과적으로 공헌 가능한 순서대로 제시해주세요: {idea}",
)

chain2 = LLMChain(llm=llm, prompt=prompt)
plan = chain2.run(idea)

print(plan)

In [ ]:
prompt = PromptTemplate(
   input_variables=["plan"],
   template="제시된 다섯 개의 계획 중에 첫 번째 계획만 쪼개서 기획서의 형태로 구성하고, 상세 내용을 포함해서 제시해주세요: {plan}"
)

chain3 = LLMChain(llm=llm, prompt=prompt)
proposal = chain3.run(plan)

print(proposal)

## SimpleSequentialChain으로 한 번에 연결하기

In [ ]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[chain, chain2, chain3], verbose=True)
print(overall_chain("농업"))

## SequentialChain으로 여러 개의 결과 얻기

In [ ]:
from langchain.chains import SequentialChain
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI()

# 아이디어 생성 (LLMChain)
template = "다음에 제시한 테마의 가치를 정의하고 새로운 비즈니스의 가능성을 개념적으로 800자 이내로 생각해주세요: {job}"
prompt_template = PromptTemplate(input_variables=["job"], template=template)
idea_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="idea")

# 아이디어를 기반으로 계획 생성 (LLMChain)
template="다음 아이디어로 구체적인 계획을 다섯 개 생각하고, 각 계획별로 세부 실행 사항을 나열하되, IT가 효과적으로 공헌 가능한 순서대로 제시해주세요: {idea}"
prompt_template = PromptTemplate(input_variables=["idea"], template=template)
plan_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="plan")

# 아이디어와 계획의 두 개의 입력을 받아 첫 번째 계획의 계획서 작성
template = "제시된 다섯 개의 계획 중에 첫 번째 계획만 쪼개서 기획서의 형태로 구성하고, 상세 내용을 포함해서 제시해주세요: {plan}"
prompt_template = PromptTemplate(input_variables=["plan"], template=template)
proposal_chain_1 = LLMChain(llm=llm, prompt=prompt_template, output_key="proposal_1")

# 아이디어와 계획의 두 개의 입력을 받아 두 번째 계획의 계획서 작성
template = "제시된 다섯 개의 계획 중에 두 번째 계획만 쪼개서 기획서의 형태로 구성하고, 상세 내용을 포함해서 제시해주세요: {plan}"
prompt_template = PromptTemplate(input_variables=["plan"], template=template)
proposal_chain_2 = LLMChain(llm=llm, prompt=prompt_template, output_key="proposal_2")

# 이 네 개의 체인을 순차적으로 실행하는 전체적인 체인
overall_chain = SequentialChain(
    chains=[idea_chain, plan_chain, proposal_chain_1,proposal_chain_2],
    input_variables=["job"],
    output_variables=["idea", "plan", "proposal_1", "proposal_2"],
    verbose=True,
)

# 최적화 후 결과 반환
results = overall_chain("농업")

# 결과 출력
print(results)

In [ ]:
print(results['proposal_1'])